The `statsmodels` implementation does not improve the model but is more convenient to use. There is no improvement, because the model is still a linear regression. The way to evaluate the parameters and find the optimum is different. The model could be improved with another technics, the polynomial regression.

In [1]:
import numpy as np
import statsmodels.api as sm
from pandas import *
from ggplot import *

We will use the same $R^2$ funtion to evaluate the model:

In [2]:
def compute_r_squared(data, predictions):
    r_squared = 1 - np.square(data - predictions).sum() / np.square(data - data.mean()).sum()
    return r_squared

It is now time to import the data:

In [3]:
turnstile_weather = pandas.read_csv('data/turnstile_data_master_with_weather.csv')
features = turnstile_weather[['rain', 'precipi', 'Hour', 'mintempi', 'fog']]

The square of the features will appended to the features, and the OLS optimisation run on this new input:

In [4]:
features_square = np.square(features)
features_square.rename(columns=lambda x: 's_' + x, inplace=True)
features = features.join(features_square)

The next steps are exactly the same than the previous exercise, dummies columns are added and the new features array is normalized:

In [5]:
dummy_units = pandas.get_dummies(turnstile_weather['UNIT'], prefix='unit')
features = features.join(dummy_units)
values = turnstile_weather['ENTRIESn_hourly']

In [6]:
def normalize_features(df):
    mu = df.mean()
    sigma = df.std()    
    if (sigma == 0).any():
        raise Exception("One or more features had the same value for all samples, and thus could " + \
                         "not be normalized. Please do not include features with only a single value " + \
                         "in your model.")
    df_normalized = (df - df.mean()) / df.std()
    return df_normalized, mu, sigma
features, mu, sigma = normalize_features(features)

In [7]:
m = len(values)
features['ones'] = np.ones(m) # Add a column of 1s (y intercept) 
features_array = np.array(features)
values_array = np.array(values)  

In [8]:
model = sm.OLS(values_array, features_array)
theta_statsmodels = model.fit().params
prediction = np.dot(features_array, theta_statsmodels)    

In [9]:
print compute_r_squared(values_array, prediction)

0.459142837857


With the polynomial regression, the $R^2$ is now better, but the improvement is very small.